In [7]:
from __future__ import print_function
import numpy

from nltk.stem import WordNetLemmatizer
import tempfile
import string

import sys
import logging
import os
import pickle
import re

# Using Gensim

In [9]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [10]:
# load the data

filename = 'df_diag.pkl'
response_folder = '~/Documents/Metis/project4'
response_folder = os.path.expanduser(response_folder)

with open(os.path.join(response_folder, filename), 'rb') as f:
    df = pickle.load(f)
df.columns = ['diag']

In [11]:
# data cleaning separate the staging direction with conversation
def sepExp(text):
    regex = re.compile(".*?\((.*?)\)")
    expression = re.findall(regex, text)
    expression = ','.join(expression)
    diag_filt = re.sub("[\(\[].*?[\)\]]", "", text)

    return expression, diag_filt

In [12]:
df['expression'], df['diag_filt'] = zip(*df['diag'].map(sepExp))

In [15]:
import nltk
from nltk.corpus import stopwords
from IPython.core.debugger import set_trace

nltk.download('wordnet')
stop = set(stopwords.words('english'))

def process(text):
    text = text.lower()
    text_filt = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    text_filt_le = [wordnet_lemmatizer.lemmatize(i) for i in text_filt]
    words_filt = [w for w in text_filt_le if w not in stop]
    return words_filt

df['tokens'] = df['diag_filt'].apply(process)
print(df.tokens)

[nltk_data] Downloading package wordnet to /Users/xuanqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
phoebe:      [wait, doe, eat, chalk, cause, want, went, car...
rachel:      [god, monica, thank, god, went, building, guy,...
ross:        [feel, like, someone, reached, throat, grabbed...
monica:      [nothing, tell, guy, work, okay, everybody, re...
chandler:    [right, joey, nice, doe, hump, hump, hairpiece...
joey:        [mon, going, guy, gotta, something, wrong, ins...
Name: tokens, dtype: object


In [16]:
# prepare the corpus

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=3, no_above=0.5):
    print('Building dictionary...')
    dictionary = Dictionary(docs)
#     print(dictionary)
    stopwords = nltk_stopwords().union(additional_stopwords)
#     print(stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
#     print(stopword_ids)
    dictionary.filter_tokens(stopword_ids)
#     print(dictionary)
    dictionary.compactify()
#     print(dictionary)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    print(dictionary)
    dictionary.compactify()
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    return dictionary, corpus

In [17]:
dictionary, corpus = prep_corpus(df.tokens)


Building dictionary...
Dictionary(1171 unique tokens: ['accepted', 'accomplished', 'adopted', 'adventure', 'agrees']...)
Building corpus...


In [18]:
MmCorpus.serialize('friends.mm', corpus)
dictionary.save('friends.dict')

In [19]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=10)
                                      
lda.save('friends.model')

CPU times: user 266 ms, sys: 39.4 ms, total: 305 ms
Wall time: 264 ms


In [20]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [446]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

In [448]:
# save the results to html file
friend_lda = open(os.path.join(response_folder, 'friend_lda.html'), 'w')
pyLDAvis.save_html(vis_data,friend_lda)

In [378]:
%%time
# The optional parameter T here indicates that HDP should find no more than 50 topics
# if there exists any.
hdp = models.hdpmodel.HdpModel(corpus, dictionary, T=50)
                                      
hdp.save('newsgroups_hdp.model')


CPU times: user 390 ms, sys: 56.6 ms, total: 447 ms
Wall time: 435 ms


In [365]:
vis_data = gensimvis.prepare(hdp, corpus, dictionary)
pyLDAvis.display(vis_data)

In [381]:

# dct = Dictionary(df.tokens)  # fit dictionary

# doc1 = dct.doc2bow(df.tokens[1])

In [397]:
# lda.get_document_topics(doc1,per_word_topics=True)

In [277]:
dictionary_0 = Dictionary(df.tokens)

# Add season in this content

In [503]:
import os
from os import listdir
from os.path import isfile, join
import glob

response_folder = '~/Documents/Metis/project4'
response_folder = os.path.expanduser(response_folder)
comp_files = [f for f in listdir(response_folder) if isfile(join(response_folder, f))]

df_diag=pd.DataFrame()
count=0
for filename in glob.glob(os.path.join(response_folder,'chapter*.pkl')):
    #print(filename)
    with open(filename,'rb') as f:
        df_temp = pickle.load(f)
        
        df_diag = df_diag.append(df_temp.loc['rachel:'])
        #print(len(df_movie))

In [505]:
df_diag.reset_index(inplace=True)
df_diag.set_index('season',inplace=True)

In [512]:
def sepExp(text):
    regex = re.compile(".*?\((.*?)\)")
    expression = re.findall(regex, text)
    expression = ','.join(expression)
    diag_filt = re.sub("[\(\[].*?[\)\]]", "", text)

    return expression, diag_filt


In [513]:
df_diag['expression'], df_diag['diag_filt'] = zip(*df_diag['diag'].map(sepExp))

In [516]:
import nltk
from nltk.corpus import stopwords
from IPython.core.debugger import set_trace

nltk.download('wordnet')
stop = set(stopwords.words('english'))

def process(text):
    text = text.lower()
    text_filt = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    text_filt_le = [wordnet_lemmatizer.lemmatize(i) for i in text_filt]
    words_filt = [w for w in text_filt_le if i not in stop]
    return words_filt
df_diag['tokens'] = df_diag['diag_filt'].apply(process)
print(df_diag.tokens)

[nltk_data] Downloading package wordnet to /Users/xuanqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
season
4.0     [nooo, way, the, most, romantic, song, ever, t...
3.0     [god, god, excuse, move, move, emergency, excu...
2.0     [god, monica, thank, god, just, went, your, bu...
6.0     [said, rachel, right, you, think, should, ther...
5.0     [you, welcome, million, god, whoa, what, why, ...
7.0     [well, hello, rachel, wait, okay, monica, and,...
8.0     [yeah, pheebs, honey, she, just, got, engaged,...
10.0    [joey, god, okay, down, the, hall, rachel, roo...
9.0     [god, who, for, you, thank, you, for, doing, t...
11.0    [ooh, lovebarbados, and, rachel, are, kissing,...
Name: tokens, dtype: object


In [507]:
df_season.head()

0     Nooo Way! The most romantic song ever is The ...
1     Oh my god. Oh my god.(She decides to make a b...
2     Oh God Monica hi! Thank God! I just went to y...
3    (To the woman sitting in front of her) He-he s...
4     Oh, you’re welcome a million.,(softly) Oh my ...
Name: diag, dtype: object

In [517]:
# cleaning lemmatizer in NLTK 

nltk.download('wordnet')
def lemmatization(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    word_new = [wordnet_lemmatizer.lemmatize(i) for i in test_filt]
    return word_new

test_filt_le = lemmatization(test_filt)
print(len(test_filt_le))
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

words_filt = [i for i in test_filt_le if i not in stop]
print(len(words_filt))

[nltk_data] Downloading package wordnet to /Users/xuanqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
21168
19088


In [560]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=2, no_above=0.4):
    print('Building dictionary...')
    dictionary = Dictionary(docs)
#     print(dictionary)
    stopwords = nltk_stopwords().union(additional_stopwords)
#     print(stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
#     print(stopword_ids)
    dictionary.filter_tokens(stopword_ids)
#     print(dictionary)
    dictionary.compactify()
#     print(dictionary)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    print(dictionary)
    dictionary.compactify()
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    return dictionary, corpus

In [574]:
dictionary, corpus = prep_corpus(df_diag.tokens)

Building dictionary...
Dictionary(1301 unique tokens: ['accidentally', 'acting', 'admit', 'adorable', 'agent']...)
Building corpus...


In [575]:
MmCorpus.serialize('rachel_season.mm', corpus)
dictionary.save('rachel_season.dict')

In [578]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=50)
                                      
lda.save('rachel_season.model')

CPU times: user 1.15 s, sys: 63.4 ms, total: 1.22 s
Wall time: 1.16 s


In [1]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

NameError: name 'gensimvis' is not defined

In [581]:
rachel_lda = open(os.path.join(response_folder, 'rachel_lda.html'), 'w')
pyLDAvis.save_html(vis_data,rachel_lda)